In [1]:
!pip install --upgrade azureml-sdk

In [2]:
%%writefile score.py
import joblib
import json
import numpy as np
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path("mnist-mlp")  # Load registered model
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = np.array(json.loads(raw_data)["data"])  # Parse input
        predictions = model.predict(data)  # Make predictions
        return {"predictions": predictions.tolist()}
    except Exception as e:
        return {"error": str(e)}


Overwriting score.py


In [6]:
%%writefile environment.yml
name: mnist-env
dependencies:
  - python=3.10
  - scikit-learn
  - joblib
  - pip
  - pip:
      - azure-ai-ml  # Required for SDK v2
      - azureml-defaults  # ✅ Required for deployment
      - azureml-inference-server-http  # ✅ Required for scoring
      - fastapi  # ✅ Required for API
      - uvicorn  # ✅ Required for API server



Overwriting environment.yml


In [7]:
!conda env remove --name mnist-env

In [8]:
!conda env create -f environment.yml

Channels:
 - defaults
Platform: linux-64
Solving environment: \ | done

mkl-2023.1.0         | 171.5 MB  |                                       |   0% 
python-3.10.16       | 26.9 MB   |                                       |   0% 

scipy-1.15.1         | 24.4 MB   |                                       |   0% 


intel-openmp-2023.1. | 17.2 MB   |                                       |   0% 



scikit-learn-1.6.1   | 9.4 MB    |                                       |   0% 




numpy-base-2.0.1     | 8.0 MB    |                                       |   0% 





libgcc-ng-11.2.0     | 5.3 MB    |                                       |   0% 






openssl-3.0.16       | 5.2 MB    |                                       |   0% 







libstdcxx-ng-11.2.0  | 4.7 MB    |                                       |   0% 








tk-8.6.14            | 3.4 MB    |                                       |   0% 









pip-25.0             | 2.3 MB    |                                   

In [10]:
!conda init


no change     /anaconda/condabin/conda
no change     /anaconda/bin/conda
no change     /anaconda/bin/conda-env
no change     /anaconda/bin/activate
no change     /anaconda/bin/deactivate
no change     /anaconda/etc/profile.d/conda.sh
no change     /anaconda/etc/fish/conf.d/conda.fish
no change     /anaconda/shell/condabin/Conda.psm1
no change     /anaconda/shell/condabin/conda-hook.ps1
no change     /anaconda/lib/python3.12/site-packages/xontrib/conda.xsh
no change     /anaconda/etc/profile.d/conda.csh
no change     /home/azureuser/.bashrc
No action taken.


In [11]:
!conda activate mnist-env


CondaError: Run 'conda init' before 'conda activate'



In [12]:
!pip list | grep azure


azure-appconfiguration                  1.1.1
azure-batch                             14.2.0
azure-cli                               2.64.0
azure-cli-core                          2.64.0
azure-cli-telemetry                     1.1.0
azure-common                            1.1.28
azure-core                              1.30.2
azure-cosmos                            3.2.0
azure-data-tables                       12.4.0
azure-datalake-store                    0.0.53
azure-graphrbac                         0.60.0
azure-identity                          1.17.1
azure-keyvault-administration           4.4.0b2
azure-keyvault-certificates             4.7.0
azure-keyvault-keys                     4.8.0
azure-keyvault-secrets                  4.7.0
azure-mgmt-advisor                      9.0.0
azure-mgmt-apimanagement                4.0.0
azure-mgmt-appconfiguration             3.0.0
azure-mgmt-appcontainers                2.0.0
azure-mgmt-applicationinsights          1.0.0
azure-mgmt-authorizatio

In [13]:
!python score.py


In [16]:
%%writefile deploy.py
from azureml.core import Workspace, Model, Environment
from azureml.core.model import InferenceConfig  # ✅ Import from correct module
from azureml.core.webservice import AciWebservice, Webservice

# Load Azure ML Workspace
ws = Workspace.from_config()

# Load Registered Model
model = Model(ws, name="mnist-mlp")

# Create an Environment
env = Environment.from_conda_specification(name="mnist-env", file_path="environment.yml")

# Create Inference Config
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# Deploy to ACI
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                        name="mnist-serviceeee",
                        models=[model],
                        inference_config=inference_config,
                        deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)

print(f"Service state: {service.state}")
print(f"Scoring URI: {service.scoring_uri}")


Overwriting deploy.py


In [17]:
!python deploy.py

/mnt/batch/tasks/shared/LS_root/mounts/clusters/janak1/code/Users/janak/deploy.py:20: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-03-22 11:31:30+00:00 Creating Container Registry if not exists.
2025-03-22 11:31:30+00:00 Registering the environment.
2025-03-22 11:31:33+